# Explo sélection de CC dans les pages outils

**Définition** : 
faire une analyse one shot de :
- taux de sélection de CC dans les outil
- taux de sortie à l’étape CC 

**Formule** :
- taux de sélection de CC dans les outil (CC select + entreprise selct) / (ccn + info_cc)
- taux de sortie à l’étape CC 

L'idée pour ces 2 points serait de comparer les résultats avant le 28/02/22 et après le 12/03/22 car le nouveau parcours a été mis en ligne le 28 février mais entre le 28 février et le 11 mars il y avait une erreur dans les event 

Pour analyser l'impact du nouveau parcours à l'étape CC qui a été mis en place le 12/03/22

In [ ]:
import pandas as pd
import time
import json
import math
import numpy as np
#from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
from query_elastic_search import load_file_if_exists_or_execute_query

In [ ]:
QUERY = {
    "query": {
        "bool": { 
          "must": [
            {
              "prefix": {
                "url": "https://code.travail.gouv.fr/outils/" 
              }
            },
     #       {
     #         "range": {
     #          "lastActionDateTime": {
     #             "gte": "2022-04-01 00:00:00",
     #             "lt": "2022-05-01 00:00:00"
     #          }
     #         }
     #       },
            {
              "range": {
               "logfile": {
                  "gte": "2022-06-01",
                  "lt": "2022-07-01"
               }
              }
            }
          ]
        }
      }
}

In [ ]:
logs_june = load_file_if_exists_or_execute_query("", QUERY)

logs_contrib.to_csv('12-21_to_05-22_logs.csv')

In [ ]:
## on peut utiliser ce jeu de données si déjà DL via la commande retrieve de monolog
from join_multiple_file import pd_read_pattern, PATH_CDTN_MONOLOG

logs_january = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-january/*")
logs_february = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-february/*")
logs_march = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-march/*")
logs_april = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-april/*")
logs_may = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-may/*")
logs_june = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-june/*")

## Filtrer par logs dans les outils et retirer l'outil dont l'url fini par /outils/convention-collective


In [ ]:
def filter_by_url(df):
    return df[ 
        (df['url'].str.startswith('https://code.travail.gouv.fr/outils/', na=False)) &
        (~df['url'].str.startswith('https://code.travail.gouv.fr/outils/convention-collective', na=False))
    ].copy() 

In [ ]:
def clean_column_url(df):
    return df['url'].str.split('#').str[0].str.split('?').str[0]

In [ ]:
logs_january_filtered = filter_by_url(logs_january)
logs_february_filtered = filter_by_url(logs_february)
logs_march_filtered = filter_by_url(logs_march)
logs_april_filtered = filter_by_url(logs_april)
logs_may_filtered = filter_by_url(logs_may)
logs_june_filtered = filter_by_url(logs_june)

logs_january_filtered['url'] = clean_column_url(logs_january_filtered)
logs_february_filtered['url'] = clean_column_url(logs_february_filtered)
logs_march_filtered['url'] = clean_column_url(logs_march_filtered)
logs_april_filtered['url'] = clean_column_url(logs_april_filtered)
logs_may_filtered['url'] = clean_column_url(logs_may_filtered)
logs_june_filtered['url'] = clean_column_url(logs_june_filtered)

## Analysons les données de janvier et févier :

FORMULE POUR LA PERIODE AVANT LE 28/02/22 (même filtres qu'avant) :
CC select / (info_cc + CCN)

In [ ]:
# nombre de visiteurs en janvier et février
pd.concat([logs_january_filtered, logs_february_filtered])['idVisit'].nunique()

Compter le taux de sélection de CC dans les outils 
: avant mars

In [ ]:
def compute_cc_select_rate_before_march(df):
    df_wout_dupl = df.drop_duplicates(subset=['idVisit', 'url', 'type', 'outilEvent']).copy()
    #df_wout_dupl = df_wout_dupl[df_wout_dupl['url']=="https://code.travail.gouv.fr/outils/preavis-demission"]
    denominator = df_wout_dupl.outilEvent.value_counts()['info_cc'] + df_wout_dupl.outilEvent.value_counts()['ccn']
    numerator = df_wout_dupl.type.value_counts()['cc_select']
    return round(numerator / denominator * 100, 1)

In [ ]:
f"{compute_cc_select_rate_before_march(logs_january_filtered)} %"

In [ ]:
f"{compute_cc_select_rate_before_march(logs_february_filtered)} %"

In [ ]:
f"{compute_cc_select_rate_before_march(pd.concat([logs_january_filtered, logs_february_filtered]))} %"

## Analysons les données pour la période avril mai juin
FORMULE POUR LA PERIODE DU 20/03/22 A AJD :
(CC SELECT P1 + ENTREPRISE SELECT) / (info_CC + CNN)

Compter le taux de sélection de CC dans les outils 
: en avril

In [ ]:
def compute_cc_select_rate_after_march(df):
    df_wout_dupl = df.drop_duplicates(subset=['idVisit', 'url', 'type', 'outilEvent']).copy()
    #df_wout_dupl = df_wout_dupl[df_wout_dupl['url']=="https://code.travail.gouv.fr/outils/preavis-demission"]
    denominator = df_wout_dupl.outilEvent.value_counts()['info_cc'] + df_wout_dupl.outilEvent.value_counts()['ccn']
    numerator = df_wout_dupl.type.value_counts()['cc_select_p1'] + df_wout_dupl.type.value_counts()['enterprise_select']
    return round(numerator / denominator * 100, 1)


In [ ]:
f"{compute_cc_select_rate_after_march(logs_april_filtered)} %"

In [ ]:
f"{compute_cc_select_rate_after_march(logs_may_filtered)} %"

In [ ]:
f"{compute_cc_select_rate_after_march(logs_june_filtered)} %"

In [ ]:
f"{compute_cc_select_rate_after_march(pd.concat([logs_april_filtered, logs_may_filtered]))} %"

In [ ]:
months = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin']
cc_select_rate = []
for df in [logs_january_filtered, logs_february_filtered]:
    cc_select_rate.append(compute_cc_select_rate_before_march(df))

begin_march = compute_cc_select_rate_after_march(logs_march_filtered[logs_march_filtered['logfile']<'2022-03-20'])
end_march = compute_cc_select_rate_after_march(logs_march_filtered[logs_march_filtered['logfile']>'2022-03-20'])
cc_select_rate.append((begin_march+end_march)/2)

for df in [logs_april_filtered, logs_may_filtered, logs_june_filtered]:
    cc_select_rate.append(compute_cc_select_rate_after_march(df))

In [ ]:
cc_select_rate

### Visualisation

In [ ]:
fig = px.line(x=months, y=cc_select_rate, markers=True, 
              labels=dict(x="Mois", y="%"))
fig.update_layout(
    title='Percentage du taux de sélection de CC dans les outils'
)
fig.update_yaxes(range=[0, 100])
fig.show()

## Part 2 : explo du taux de sortie à l'étape CC : 

**Formule** : nombre de visites à l'étape info_CC ou CNN - nombre de visites à l'étape d'après (différente en fonction de l'outil)

 

In [ ]:
DICT_URL_TOOLS_STEPS = {
  'https://code.travail.gouv.fr/outils/preavis-retraite': {
    'cc_step': 'ccn',
    'next_step': 'infos',
  },
  'https://code.travail.gouv.fr/outils/indemnite-precarite': {
    'cc_step': 'info_cc',
    'next_step': 'info_generales',
  },
  'https://code.travail.gouv.fr/outils/preavis-licenciement': {
    'cc_step': 'info_cc',
    'next_step': 'results',
  }
}

In [ ]:
DICT_OF_DATAFRAMES_BY_MONTH = {
  'january': logs_january_filtered,
  'february': logs_february_filtered,
  'april': logs_april_filtered,
  'may': logs_may_filtered,
  'june' : logs_june_filtered
}

In [ ]:
def get_cc_step_and_next_step(tool_steps):
    return tool_steps['cc_step'], tool_steps['next_step']

In [ ]:
def get_number_of_visitors_for_cc_step_and_next_step(df_tool, tool_steps):
    cc_step, next_step = get_cc_step_and_next_step(tool_steps)
    nb_visitors_at_cc_step = df_tool.outilEvent.value_counts()[cc_step]
    nb_visitors_at_next_step = df_tool.outilEvent.value_counts()[next_step]
    return nb_visitors_at_cc_step, nb_visitors_at_next_step

def get_exit_rate(df_tool, tool_steps):
    nb_visitors_at_cc_step, nb_visitors_at_next_step = \
        get_number_of_visitors_for_cc_step_and_next_step(df_tool, tool_steps)

    return round((nb_visitors_at_cc_step - nb_visitors_at_next_step) / nb_visitors_at_cc_step * 100, 2)

def get_acc_of_number_of_visitors_for_cc_step_and_next_step(
    df_tool, tool_steps, acc_nb_visitors_at_cc_step, acc_nb_visitors_at_next_step
):
    nb_visitors_at_cc_step, nb_visitors_at_next_step = \
        get_number_of_visitors_for_cc_step_and_next_step(df_tool, tool_steps)
    
    acc_nb_visitors_at_cc_step += nb_visitors_at_cc_step
    acc_nb_visitors_at_next_step += nb_visitors_at_next_step

    return acc_nb_visitors_at_cc_step, acc_nb_visitors_at_next_step


In [ ]:
def compute_exit_rate_before_and_after_selecting_a_cc_in_tools():
    for month, df in DICT_OF_DATAFRAMES_BY_MONTH.items():
        print(month)
        df_wout_duplicates = df.drop_duplicates(subset=['idVisit', 'url', 'outilEvent']).copy()
        acc_nb_visitors_at_cc_step, acc_nb_visitors_at_next_step = 0, 0
        
        for tool_url, tool_steps in DICT_URL_TOOLS_STEPS.items():
            df_tool = df_wout_duplicates[df_wout_duplicates['url']==tool_url]
            
            exit_rate = get_exit_rate(df_tool, tool_steps)
            
            acc_nb_visitors_at_cc_step, acc_nb_visitors_at_next_step = \
                get_acc_of_number_of_visitors_for_cc_step_and_next_step(
                df_tool, 
                tool_steps, 
                acc_nb_visitors_at_cc_step, 
                acc_nb_visitors_at_next_step
            )
            
            print(f"\t{tool_url}")
            print(f"\tTaux de sortie est : {exit_rate}%")
        print("ACC")
        acc_exit_rate = round((acc_nb_visitors_at_cc_step - acc_nb_visitors_at_next_step) / acc_nb_visitors_at_cc_step * 100, 2)
        print(f"Taux de sortie est : {acc_exit_rate}%")
        

In [ ]:
compute_exit_rate_before_and_after_selecting_a_cc_in_tools()

In [ ]:
# Taux de sortie avec une meilleure présentation (par outil puis par mois)
def compute_exit_rate_before_and_after_selecting_a_cc_in_tools():
    for tool_url, tool_steps in DICT_URL_TOOLS_STEPS.items():
        print(f"{tool_url}")
        
        for month, df in DICT_OF_DATAFRAMES_BY_MONTH.items():
            print(f"\t{month}")
            df_wout_duplicates = df.drop_duplicates(subset=['idVisit', 'url', 'outilEvent']).copy()
            df_tool = df_wout_duplicates[df_wout_duplicates['url']==tool_url]
            
            exit_rate = get_exit_rate(df_tool, tool_steps)
            print(f"\tTaux de sortie est : {exit_rate}%")
        print()

In [ ]:
compute_exit_rate_before_and_after_selecting_a_cc_in_tools()

### Pour l'outil préavis de démission, c'est un cas particulier car il faut découper les données sur avril

In [ ]:
DICT_URL_PREAVIS_DEMISSION_STEPS = {
  'https://code.travail.gouv.fr/outils/preavis-demission': {
    'cc_step': 'info_cc',
    'next_step': 'results',
  }
}

In [ ]:
DICT_OF_DATAFRAMES_BY_MONTH_FOR_PREAVIS_DEMISSION = {
  'january': logs_january_filtered,
  'february': logs_february_filtered,
  'april': logs_april_filtered[logs_april_filtered['logfile']<'2022-04-27'],
  'may': logs_may_filtered,
  'june' : logs_june_filtered
}

In [ ]:
def compute_exit_rate_before_and_after_selecting_a_cc_in_preavis_demission():
    for tool_url, tool_steps in DICT_URL_PREAVIS_DEMISSION_STEPS.items():
        print(f"{tool_url}")
        
        for month, df in DICT_OF_DATAFRAMES_BY_MONTH_FOR_PREAVIS_DEMISSION.items():
            print(f"\t{month}")
            df_wout_duplicates = df.drop_duplicates(subset=['idVisit', 'url', 'outilEvent']).copy()
            df_tool = df_wout_duplicates[df_wout_duplicates['url']==tool_url]
            
            if month == 'may':
                tool_steps['next_step'] = 'infos'
            exit_rate = get_exit_rate(df_tool, tool_steps)
            print(f"\tTaux de sortie est : {exit_rate}%")
        print()

In [ ]:
compute_exit_rate_before_and_after_selecting_a_cc_in_preavis_demission()

In [ ]:
logs_may_filtered.url.value_counts()[:20]

In [ ]:
logs_may_filtered[
    logs_may_filtered['url']=="https://code.travail.gouv.fr/outils/preavis-retraite"
].outilEvent.value_counts()

## Partie 3

KPI : Pourcentage de personnes qui obtiennent un résultat conventionnel dans les outils

**Définition** : Part des personnes qui vont jusqu'à l'étape résultat des outils et qui ont sélectionné une CC traitée par rapport au nombre total de personnes qui vont jusqu'à l'étape résultat des outils

**Formule** : (Nb de sessions allant jusqu'à l'étape résultat + CC select)/ (Nb de sessions allant jusqu'à l'étape résultat)*100

In [ ]:
DICT_URL_TOOLS_STEPS = {
    'https://code.travail.gouv.fr/outils/heures-recherche-emploi': 'results',
    'https://code.travail.gouv.fr/outils/indemnite-precarite': 'indemnite',
    'https://code.travail.gouv.fr/outils/preavis-demission': 'results',
    'https://code.travail.gouv.fr/outils/preavis-licenciement': 'results',
    'https://code.travail.gouv.fr/outils/preavis-retraite': 'result',
}
DICT_OF_DATAFRAMES_BY_MONTH = {
  'april': logs_april_filtered,
  #'may':   logs_may_filtered,
  #'june':  logs_june_filtered
}

In [ ]:
for url_tool, last_step in DICT_URL_TOOLS_STEPS.items():
    print(url_tool)
    
    for month, df in DICT_OF_DATAFRAMES_BY_MONTH.items():
        print(month)
        df_tool = df[df['url'] == url_tool]
        
        all_visitors_reaching_result_step = df_tool[df_tool['outilEvent'] == last_step].idVisit.unique()
        nb_visitors_reaching_result_step = len(all_visitors_reaching_result_step)
        
        all_visitors_who_have_selected_cc_treated = df_tool[df_tool['type'] == 'cc_select_traitée'].idVisit.unique()
        nb_visitors_who_have_selected_cc_treated_n_reaching_result_step = len(
            set(all_visitors_who_have_selected_cc_treated) & set(all_visitors_reaching_result_step))
        
        print(f"\t{nb_visitors_reaching_result_step}")
        print(f"\t{nb_visitors_who_have_selected_cc_treated_n_reaching_result_step}")
        
        print(f"{round(nb_visitors_who_have_selected_cc_treated_n_reaching_result_step / nb_visitors_reaching_result_step * 100, 2)} %")
        

In [ ]:
for url_tool, last_step in DICT_URL_TOOLS_STEPS.items():
    print(url_tool)
    
    for month, df in DICT_OF_DATAFRAMES_BY_MONTH.items():
        print(month)
        df_tool = df[df['url'] == url_tool]
        
        all_visitors_reaching_result_step = df_tool[df_tool['outilEvent'] == last_step].idVisit.unique()
        nb_visitors_reaching_result_step = len(all_visitors_reaching_result_step)
        
        all_visitors_who_have_selected_cc = df_tool[
            df_tool['type'].isin(['cc_select_p1', 'enterprise_select'])].idVisit.unique()
        
        all_visitors_who_have_selected_cc_treated = df_tool[
            df_tool['type'] == 'cc_select_traitée'].idVisit.unique()
        
        nb_visitors_who_have_selected_cc_treated_n_reaching_result_step = len(
            set(all_visitors_who_have_selected_cc) & 
            set(all_visitors_who_have_selected_cc_treated) & 
            set(all_visitors_reaching_result_step)
        )
        #print(nb_visitors_reaching_result_step)
        #print(nb_visitors_who_have_selected_cc_treated_n_reaching_result_step)
        ratio = nb_visitors_who_have_selected_cc_treated_n_reaching_result_step / nb_visitors_reaching_result_step
        print(f"{round(ratio * 100, 2)} %")
        

In [ ]:
for url_tool, last_step in DICT_URL_TOOLS_STEPS.items():
    print(url_tool)
    
    for month, df in DICT_OF_DATAFRAMES_BY_MONTH.items():
        print(month)
        df_tool = df[df['url'] == url_tool]
        
        all_visitors_reaching_result_step = df_tool[df_tool['outilEvent'] == last_step].idVisit.unique()
        nb_visitors_reaching_result_step = len(all_visitors_reaching_result_step)
        
        all_visitors_who_have_selected_cc_treated = df_tool[df_tool['type'] == 'cc_select_traitée'].idVisit.unique()
        nb_visitors_who_have_selected_cc_treated_n_reaching_result_step = len(
            set(all_visitors_who_have_selected_cc_treated) & set(all_visitors_reaching_result_step))
        
        ratio1 = nb_visitors_who_have_selected_cc_treated_n_reaching_result_step / nb_visitors_reaching_result_step

        
        all_visitors_who_have_selected_cc = df_tool[
            df_tool['type'].isin(['cc_select_p1', 'enterprise_select'])].idVisit.unique()
        
        all_visitors_who_have_selected_cc_treated = df_tool[
            df_tool['type'] == 'cc_select_traitée'].idVisit.unique()
        
        nb_visitors_who_have_selected_cc_treated_n_reaching_result_step = len(
            set(all_visitors_who_have_selected_cc) & 
            set(all_visitors_who_have_selected_cc_treated) & 
            set(all_visitors_reaching_result_step)
        )
        #print(nb_visitors_reaching_result_step)
        #print(nb_visitors_who_have_selected_cc_treated_n_reaching_result_step)
        ratio2 = nb_visitors_who_have_selected_cc_treated_n_reaching_result_step / nb_visitors_reaching_result_step
        print(f"{round(abs(ratio1 - ratio2) * 100, 2)} %")
        